In [43]:
import requests
import json
from pandas.io.json import json_normalize
import time
import pandas as pd
import numpy as np

In [23]:
r = requests.get('https://lab.isaaclin.cn/nCoV/api/overall',{'latest':0})

In [9]:
results = json.loads(r.text)['results']

In [12]:
df = json_normalize(results)

In [18]:
df.tail()

,abroadRemark,confirmedCount,confirmedIncr,curedCount,curedIncr,currentConfirmedCount,currentConfirmedIncr,deadCount,deadIncr,generalRemark,...,remark1,remark2,remark3,remark4,remark5,seriousCount,seriousIncr,suspectedCount,suspectedIncr,updateTime
1645,NaN,894,NaN,35,NaN,NaN,NaN,26,NaN,NaN,...,病毒是否变异：存在可能,疫情是否扩散：是,,,,NaN,NaN,1076,NaN,1579866640114
1646,NaN,887,NaN,35,NaN,NaN,NaN,26,NaN,NaN,...,病毒是否变异：存在可能,疫情是否扩散：是,,,,NaN,NaN,1076,NaN,1579864394844
1647,NaN,887,NaN,35,NaN,NaN,NaN,26,NaN,NaN,...,病毒是否变异：存在可能,疫情是否扩散：是,,,,NaN,NaN,1076,NaN,1579860573823
1648,NaN,887,NaN,35,NaN,NaN,NaN,26,NaN,NaN,...,病毒是否变异：存在可能,疫情是否扩散：是,,,,NaN,NaN,1075,NaN,1579857967369
1649,NaN,887,NaN,35,NaN,NaN,NaN,26,NaN,NaN,...,病毒是否变异：存在可能,疫情是否扩散：是,,,,NaN,NaN,1075,NaN,1579854581322


In [17]:
df.columns

Index(['abroadRemark', 'confirmedCount', 'confirmedIncr', 'curedCount',
       'curedIncr', 'currentConfirmedCount', 'currentConfirmedIncr',
       'deadCount', 'deadIncr', 'generalRemark', 'note1', 'note2', 'note3',
       'remark1', 'remark2', 'remark3', 'remark4', 'remark5', 'seriousCount',
       'seriousIncr', 'suspectedCount', 'suspectedIncr', 'updateTime'],
      dtype='object')

In [61]:
data = df.reindex(columns=['confirmedCount','currentConfirmedCount','curedCount', 'deadCount', 'updateTime'])

In [62]:
data.head()

,confirmedCount,currentConfirmedCount,curedCount,deadCount,updateTime
0,77782,47589.0,27527,2666,1582620948850
1,77782,47616.0,27500,2666,1582620572458
2,77782,47617.0,27499,2666,1582620445147
3,77782,47630.0,27486,2666,1582618996656
4,77782,47631.0,27485,2666,1582618682516


In [63]:
def time_format(timeNum):
    timeTemp = float(timeNum/1000)
    tupTime = time.localtime(timeTemp)
    stadardTime = time.strftime("%Y-%m-%d %H:%M:%S", tupTime)
    return stadardTime

In [64]:
data['updateTime'] = data['updateTime'].map(time_format)

In [65]:
time_full = data['updateTime'].str.split(' ', expand=True).rename(columns={0:'date',1:'time'})

In [66]:
data_full = pd.concat([data,time_full],axis=1)

In [67]:
data_full.head()

,confirmedCount,currentConfirmedCount,curedCount,deadCount,updateTime,date,time
0,77782,47589.0,27527,2666,2020-02-25 16:55:48,2020-02-25,16:55:48
1,77782,47616.0,27500,2666,2020-02-25 16:49:32,2020-02-25,16:49:32
2,77782,47617.0,27499,2666,2020-02-25 16:47:25,2020-02-25,16:47:25
3,77782,47630.0,27486,2666,2020-02-25 16:23:16,2020-02-25,16:23:16
4,77782,47631.0,27485,2666,2020-02-25 16:18:02,2020-02-25,16:18:02


In [68]:
def top(df, n=1, column='time'):
    return df.sort_values(by=column)[-n:]

In [70]:
data_top = data_full.groupby('date').apply(top)

In [74]:
data_top['currentConfirmedCount'] = data_top['confirmedCount']-data_top['curedCount']-data_top['deadCount']

In [75]:
data_top

,,confirmedCount,currentConfirmedCount,curedCount,deadCount,updateTime,date,time
date,,,,,,,,
2020-01-24,1641,897,835,36,26,2020-01-24 23:54:14,2020-01-24,23:54:14
2020-01-25,1614,1408,1327,39,42,2020-01-25 23:54:35,2020-01-25,23:54:35
2020-01-26,1589,2076,1971,49,56,2020-01-26 23:03:31,2020-01-26,23:03:31
2020-01-27,1554,2857,2719,56,82,2020-01-27 23:55:25,2020-01-27,23:55:25
2020-01-28,1532,4630,4451,73,106,2020-01-28 21:02:03,2020-01-28,21:02:03
2020-01-29,1490,6095,5843,119,133,2020-01-29 23:31:04,2020-01-29,23:31:04
2020-01-30,1465,8149,7843,135,171,2020-01-30 23:08:59,2020-01-30,23:08:59
2020-01-31,1437,9811,9384,214,213,2020-01-31 22:43:05,2020-01-31,22:43:05
2020-02-01,1400,11901,11367,275,259,2020-02-01 23:46:53,2020-02-01,23:46:53


In [91]:
currentConfirmedCount = data_top[['currentConfirmedCount','date']].rename(columns={'currentConfirmedCount':'value'})
currentConfirmedCount['name'] = '现存确诊'
curedCount = data_top[['curedCount','date']].rename(columns={'curedCount':'value'})
curedCount['name'] = '累计治愈'
deadCount = data_top[['deadCount','date']].rename(columns={'deadCount':'value'})
deadCount['name'] = '累计死亡'

In [94]:
data_out = pd.concat([currentConfirmedCount,curedCount,deadCount],ignore_index=True,axis=0).reindex(columns=['name','value','date'])

In [98]:
data_out.to_csv('data_out.csv',index=False)

In [99]:
data_out

,name,value,date
0,现存确诊,835,2020-01-24
1,现存确诊,1327,2020-01-25
2,现存确诊,1971,2020-01-26
3,现存确诊,2719,2020-01-27
4,现存确诊,4451,2020-01-28
5,现存确诊,5843,2020-01-29
6,现存确诊,7843,2020-01-30
7,现存确诊,9384,2020-01-31
8,现存确诊,11367,2020-02-01
9,现存确诊,13752,2020-02-02
